https://scikit-learn.org/1.1/modules/decomposition.html#principal-component-analysis-pca

## Training and Logging

In [1]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
(X.shape, y.shape)

((150, 4), (150,))

In [2]:
[(X[0], y[0]), (X[1], y[1])]

[(array([5.1, 3.5, 1.4, 0.2]), 0), (array([4.9, 3. , 1.4, 0.2]), 0)]

In [3]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)
X_r[0]

array([-2.68412563,  0.31939725])

In [4]:
import getpass

import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.decomposition import PCA


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model = PCA(n_components=2)
    model.fit(X, y)
    registered_model_name = f"{getpass.getuser()}_pca"
    log_model(model, registered_model_name=registered_model_name)


2023/03/29 13:14:01 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
/home/da/.cache/pants/named_caches/pex_root/venvs/s/378f4125/venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/03/29 13:14:03 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_pca' already exists. Creating a new version of this model...
2023/03/29 13:14:04 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_pca, version 3
Created version '3' of model 'da_pca'.
/home/da/.cache/pants/named_caches/pex_root/venvs/s/378f4125/venv/lib/python3.9/site-packages/liga/mlflow/logger.py:137: UserWarning: value of rikai.output.schema is None or empt

## Apply the model on large scale dataset

In [5]:
from example import spark

from liga.mlflow import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m LOCATION 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


2023-03-29 13:14:04,133 INFO Rikai (__init__.py:121): setting spark.sql.extensions to net.xmacs.liga.spark.RikaiSparkSessionExtensions
2023-03-29 13:14:04,133 INFO Rikai (__init__.py:121): setting spark.driver.extraJavaOptions to -Dio.netty.tryReflectionSetAccessible=true
2023-03-29 13:14:04,133 INFO Rikai (__init__.py:121): setting spark.executor.extraJavaOptions to -Dio.netty.tryReflectionSetAccessible=true
2023-03-29 13:14:04,134 INFO Rikai (__init__.py:121): setting spark.jars to https://github.com/komprenilo/liga/releases/download/v0.3.0/liga-spark321-assembly_2.12-0.3.0.jar
23/03/29 13:14:05 WARN Utils: Your hostname, tubi resolves to a loopback address: 127.0.1.1; using 192.168.31.32 instead (on interface wlp0s20f3)
23/03/29 13:14:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For Spar

+----------------+------+----------------+-------+
|name            |plugin|uri             |options|
+----------------+------+----------------+-------+
|mlflow_sklearn_m|      |mlflow:///da_pca|       |
+----------------+------+----------------+-------+



In [6]:
from liga.numpy.sql import literal

result = spark.sql(f"""
select
  ML_PREDICT(mlflow_sklearn_m, {literal(X[0])})
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: array (nullable = true)
 |    |-- element: float (containsNull = true)



,mlflow_sklearn_m
0,"[-2.6841256618499756, 0.3193972408771515]"
